First downlaod the module pytrends.

```
# pip install pytrends

```

Cite: https://github.com/qztseng/google-trends-daily/blob/master/gtrend.py



In [2]:
!pip install pytrends

distutils: c:\users\saura\python\Include\UNKNOWN
sysconfig: c:\users\saura\python\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\saura\python\Include\UNKNOWN
sysconfig: c:\users\saura\python\Include
user = False
home = None
root = None
prefix = None


In [3]:
from pytrends.request import TrendReq
from pytrends.exceptions import ResponseError
from datetime import datetime, timedelta, date, time
import pandas as pd
import time

In [4]:
def _fetch_data(trendreq, kw_list, timeframe='today 3-m', cat=0, geo='', gprop='') -> pd.DataFrame:
    
    """Download google trends data using pytrends TrendReq and retry the same(in case of a ResponseError)."""
    attempts, fetched = 0, False
    while not fetched:
        try:
            trendreq.build_payload(kw_list=kw_list, timeframe=timeframe, cat=cat, geo=geo, gprop=gprop)
        except ResponseError as err:
            print(err)
            print(f'Trying again in {60 + 5 * attempts} seconds.')
            sleep(60 + 5 * attempts)
            attempts += 1
            if attempts > 3:
                print('Failed after 3 attemps, abort fetching.')
                break
        else:
            fetched = True
    return trendreq.interest_over_time()


In [5]:
def get_daily_trend(trendreq, keyword:str, start:str, end:str, cat=0, 
                    geo='', gprop='', delta=269, overlap=100, sleep=0, 
                    tz=0, verbose=False) ->pd.DataFrame:

    """Stich and scale consecutive daily trends data between start and end date.
    This function will first download piece-wise google trends data and then 
    scale each piece using the overlapped period. 
        Parameters
        ----------
        trendreq : TrendReq(hl='EN', tz=360)
            a pytrends TrendReq object, here hl selects language and tz is timezone offset
        keyword: str
            Enter your keyword to find trends for
        start: str
            starting date in string format:YYYY-MM-DD (e.g.2017-02-19)
        end: str
            ending date in string format:YYYY-MM-DD (e.g.2017-02-19)
        cat, geo, gprop, sleep: 
            Important is geo which means geographical area. Default is worldwide
            gprop means where to search like images, web, etc. Default is web
        delta: int
            The length(days) of each timeframe fragment for fetching google trends data, 
            need to be <269 in order to obtain daily data.
        overlap: int
            The length(days) of the overlap period used for scaling/normalization
        tz: int
            The timezone shift in minute relative to the UTC+0 (google trends default).
            For example, correcting for UTC+8 is 480, and UTC-6 is -360 
    """
    
    start_d = datetime.strptime(start, '%Y-%m-%d')
    init_end_d = end_d = datetime.strptime(end, '%Y-%m-%d')
    init_end_d.replace(hour=23, minute=59, second=59)    
    delta = timedelta(days=delta)
    overlap = timedelta(days=overlap)

    itr_d = end_d - delta
    overlap_start = None

    df = pd.DataFrame()
    ol = pd.DataFrame()
    
    while end_d > start_d:
        tf = itr_d.strftime('%Y-%m-%d')+' '+end_d.strftime('%Y-%m-%d')
        if verbose: print('Fetching \''+keyword+'\' for period:'+tf)
        temp = _fetch_data(trendreq, [keyword], timeframe=tf, cat=cat, geo=geo, gprop=gprop)
        temp.drop(columns=['isPartial'], inplace=True)
        temp.columns.values[0] = tf
        ol_temp = temp.copy()
        ol_temp.iloc[:,:] = None
        if overlap_start is not None:  # not first iteration
            if verbose: print('Normalize by overlapping period:'+overlap_start.strftime('%Y-%m-%d'), end_d.strftime('%Y-%m-%d'))
            #normalize using the maximum value of the overlapped period
            y1 = temp.loc[overlap_start:end_d].iloc[:,0].values.max()
            y2 = df.loc[overlap_start:end_d].iloc[:,-1].values.max()
            coef = y2/y1
            temp = temp * coef
            ol_temp.loc[overlap_start:end_d, :] = 1 

        df = pd.concat([df,temp], axis=1)
        ol = pd.concat([ol, ol_temp], axis=1)
        # shift the timeframe for next iteration
        overlap_start = itr_d
        end_d -= (delta-overlap)
        itr_d -= (delta-overlap)
        # in case of short query interval getting banned by server
        time.sleep(sleep)
    
    df.sort_index(inplace=True)
    ol.sort_index(inplace=True)
    #The daily trend data is missing the most recent 3-days data, need to complete with hourly data
    if df.index.max() < init_end_d : 
        tf = 'now 7-d'
        hourly = _fetch_data(trendreq, [keyword], timeframe=tf, cat=cat, geo=geo, gprop=gprop)
        hourly.drop(columns=['isPartial'], inplace=True)
        
        #convert hourly data to daily data
        daily = hourly.groupby(hourly.index.date).sum()
        
        #check whether the first day data is complete (i.e. has 24 hours)
        daily['hours'] = hourly.groupby(hourly.index.date).count()
        if daily.iloc[0].loc['hours'] != 24: daily.drop(daily.index[0], inplace=True)
        daily.drop(columns='hours', inplace=True)
        
        daily.set_index(pd.DatetimeIndex(daily.index), inplace=True)
        daily.columns = [tf]
        
        ol_temp = daily.copy()
        ol_temp.iloc[:,:] = None
        # find the overlapping date
        intersect = df.index.intersection(daily.index)
        if verbose: print('Normalize by overlapping period:'+(intersect.min().strftime('%Y-%m-%d'))+' '+(intersect.max().strftime('%Y-%m-%d')))
        # scaling use the overlapped today-4 to today-7 data
        coef = df.loc[intersect].iloc[:,0].max() / daily.loc[intersect].iloc[:,0].max()
        daily = (daily*coef).round(decimals=0)
        ol_temp.loc[intersect,:] = 1
        
        df = pd.concat([daily, df], axis=1)
        ol = pd.concat([ol_temp, ol], axis=1)

    # taking averages for overlapped period
    df = df.mean(axis=1)
    ol = ol.max(axis=1)
    # merge the two dataframe (trend data and overlap flag)
    df = pd.concat([df,ol], axis=1)
    df.columns = ['attention', 'overlap']
    # Correct the timezone difference
    df.index = df.index + timedelta(minutes=tz)
    df = df[start_d:init_end_d]
    # re-normalized to the overall maximum value to have max =100
    df['attention'] = (100*df['attention']/df['attention'].max()).round(decimals=0)

    df.drop('overlap', axis='columns', inplace=True)
    
    return df

# Making All essential data files

In [6]:
df = get_daily_trend(TrendReq(hl='EN', tz=360),'AMD', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [9]:
df.head()

,attention
date,
2016-10-01,44.0
2016-10-02,41.0
2016-10-03,38.0
2016-10-04,41.0
2016-10-05,43.0


In [10]:
df_1 = get_daily_trend(TrendReq(hl='EN', tz=360),'MU', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [11]:
df_1

,attention
date,
2016-10-01,38.0
2016-10-02,48.0
2016-10-03,33.0
2016-10-04,35.0
2016-10-05,35.0
...,...
2021-04-17,48.0
2021-04-18,56.0
2021-04-19,46.0


In [13]:
df_2 = get_daily_trend(TrendReq(hl='EN', tz=360),'AAPL', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [14]:
df_2

,attention
date,
2016-10-01,5.0
2016-10-02,3.0
2016-10-03,13.0
2016-10-04,17.0
2016-10-05,14.0
...,...
2021-04-17,6.0
2021-04-18,5.0
2021-04-19,22.0


In [15]:
df_3 = get_daily_trend(TrendReq(hl='EN', tz=360),'FB', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [16]:
df_3

,attention
date,
2016-10-01,98.0
2016-10-02,95.0
2016-10-03,86.0
2016-10-04,82.0
2016-10-05,83.0
...,...
2021-04-17,23.0
2021-04-18,24.0
2021-04-19,21.0


In [17]:
df_4 = get_daily_trend(TrendReq(hl='EN', tz=360),'SNAP', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [18]:
df_4

,attention
date,
2016-10-01,32.0
2016-10-02,36.0
2016-10-03,32.0
2016-10-04,30.0
2016-10-05,31.0
...,...
2021-04-17,31.0
2021-04-18,33.0
2021-04-19,33.0


In [19]:
df_5 = get_daily_trend(TrendReq(hl='EN', tz=360),'MSFT', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [20]:
df_5

,attention
date,
2016-10-01,2.0
2016-10-02,1.0
2016-10-03,7.0
2016-10-04,10.0
2016-10-05,5.0
...,...
2021-04-17,8.0
2021-04-18,5.0
2021-04-19,24.0


In [21]:
df_6 = get_daily_trend(TrendReq(hl='EN', tz=360),'TWTR', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [22]:
df_6

,attention
date,
2016-10-01,20.0
2016-10-02,7.0
2016-10-03,36.0
2016-10-04,33.0
2016-10-05,59.0
...,...
2021-04-17,4.0
2021-04-18,4.0
2021-04-19,5.0


In [23]:
df_7 = get_daily_trend(TrendReq(hl='EN', tz=360),'INTC', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [24]:
df_7

,attention
date,
2016-10-01,4.0
2016-10-02,3.0
2016-10-03,12.0
2016-10-04,7.0
2016-10-05,16.0
...,...
2021-04-17,3.0
2021-04-18,2.0
2021-04-19,13.0


In [25]:
df_8 = get_daily_trend(TrendReq(hl='EN', tz=360),'BABA', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [26]:
df_8

,attention
date,
2016-10-01,41.0
2016-10-02,39.0
2016-10-03,33.0
2016-10-04,34.0
2016-10-05,33.0
...,...
2021-04-17,46.0
2021-04-18,47.0
2021-04-19,41.0


In [27]:
df_9 = get_daily_trend(TrendReq(hl='EN', tz=360),'JD', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [28]:
df_9

,attention
date,
2016-10-01,25.0
2016-10-02,25.0
2016-10-03,21.0
2016-10-04,21.0
2016-10-05,20.0
...,...
2021-04-17,39.0
2021-04-18,39.0
2021-04-19,38.0


In [29]:
df.to_csv('AMD_attention.csv')
df_1.to_csv('MU_attention.csv')
df_2.to_csv('AAPL_attention.csv')
df_3.to_csv('FB_attention.csv')
df_4.to_csv('SNAP_attention.csv')
df_5.to_csv('MSFT_attention.csv')
df_6.to_csv('TWTR_aatention.csv')
df_7.to_csv('INTC_aatention.csv')
df_8.to_csv('BABA_aatention.csv')
df_9.to_csv('JD_aatention.csv')

In [30]:
df_10 = get_daily_trend(TrendReq(hl='EN', tz=360),'BAC', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_11 = get_daily_trend(TrendReq(hl='EN', tz=360),'Citigroup', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_12 = get_daily_trend(TrendReq(hl='EN', tz=360),'WFC', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_13 = get_daily_trend(TrendReq(hl='EN', tz=360),'JPM', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_14 = get_daily_trend(TrendReq(hl='EN', tz=360),'MS', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_15 = get_daily_trend(TrendReq(hl='EN', tz=360),'Blackstone Group', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_16 = get_daily_trend(TrendReq(hl='EN', tz=360),'KKR', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_17 = get_daily_trend(TrendReq(hl='EN', tz=360),'AIG', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_18 = get_daily_trend(TrendReq(hl='EN', tz=360),'SCHW', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_19 = get_daily_trend(TrendReq(hl='EN', tz=360),'QD', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [31]:
df_20 = get_daily_trend(TrendReq(hl='EN', tz=360),'RF', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_21 = get_daily_trend(TrendReq(hl='EN', tz=360),'Credit Suisse', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_22 = get_daily_trend(TrendReq(hl='EN', tz=360),'KEY', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_23 = get_daily_trend(TrendReq(hl='EN', tz=360),'WETF', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_24 = get_daily_trend(TrendReq(hl='EN', tz=360),'DB', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_25 = get_daily_trend(TrendReq(hl='EN', tz=360),'GS', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_26 = get_daily_trend(TrendReq(hl='EN', tz=360),'ALLY', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_27 = get_daily_trend(TrendReq(hl='EN', tz=360),'APO', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_28 = get_daily_trend(TrendReq(hl='EN', tz=360),'MET', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')
df_29 = get_daily_trend(TrendReq(hl='EN', tz=360),'WY', '2016-10-01', '2021-4-21', cat = 0, geo='', gprop='')

In [32]:
df_10.to_csv('BAC_attention.csv')
df_11.to_csv('C_attention.csv')
df_12.to_csv('WFC_attention.csv')
df_13.to_csv('JPM_attention.csv')
df_14.to_csv('MS_attention.csv')
df_15.to_csv('BX_attention.csv')
df_16.to_csv('KKR_attention.csv')
df_17.to_csv('AIG_attention.csv')
df_18.to_csv('SCHW_attention.csv')
df_19.to_csv('QD_attention.csv')

In [33]:
df_20.to_csv('RF_attention.csv')
df_21.to_csv('CS_attention.csv')
df_22.to_csv('KEY_attention.csv')
df_23.to_csv('WETF_attention.csv')
df_24.to_csv('DB_attention.csv')
df_25.to_csv('GS_attention.csv')
df_26.to_csv('ALLY_attention.csv')
df_27.to_csv('APO_attention.csv')
df_28.to_csv('MET_attention.csv')
df_29.to_csv('WY_attention.csv')